## ⚪ **Approche Naïve : Baseline Lexicales et Fréquentielle**

Avant d'utiliser des architectures complexes de type Transformers, nous mettons en place une approche naïve. Cette étape est fondamentale pour quantifier la difficulté de la tâche et justifier l'utilisation d'approche plus complexe.

Cette étape permet également de mesurer l'impact du Domain Shift. En appliquant un dictionnaire appris sur des essais étudiants à des résumés médicaux, nous quantifions les limites de la reconnaissance de mots-clés face à une terminologie technique inconnue. Cette baseline justifie ainsi l'utilité des modèles profonds comme RoBERTa pour capturer la sémantique là où les statistiques de mots échouent.

___
##**Préparation des Données et Apprentissage Lexicale**

Dans cette première étape, nous procédons au chargement des données et à la construction de notre modèle de référence. Cette approche est qualifiée de naïve car elle repose exclusivement sur la fréquence statistique des mots, faisant abstraction du contexte global de la phrase ou de la structure logique du discours.

Le processus débute par un chargement multi-domaine via la fonction `load_and_extract` afin de récupérer les textes et les étiquettes BIO pour le domaine source des essais et le domaine cible médical. Ensuite, nous effectuons une analyse statistique sur l'ensemble d'entraînement pour comptabiliser, pour chaque mot unique, le label qui lui est le plus souvent attribué. Enfin, nous stabilisons cette règle de décision dans un dictionnaire nommé `naive_model` qui servira de base unique pour toutes nos prédictions. Cette méthode permet de tester l'hypothèse selon laquelle certains mots seraient des indicateurs argumentatifs suffisants, indépendamment de leur position ou de la sémantique complexe de la phrase.

In [21]:
import json
from pathlib import Path
from collections import Counter
from seqeval.metrics import classification_report

data_dir = Path("data")

def load_and_extract(file_name):
    """
    Charge un fichier JSON et extrait les séquences de mots et leurs étiquettes BIO.

    Args:
        file_name (str): Nom du fichier à charger dans le dossier data.

    Returns:
        tuple: (sentences, labels) où chaque élément est une liste de listes.
    """
    path = data_dir / file_name
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    sentences, labels = [], []
    for doc in data:
        for paragraph in doc["tokens"]:
            if not paragraph: continue
            words = [token["str"] for token in paragraph]
            tags = [token["arg"] for token in paragraph]
            sentences.append(words)
            labels.append(tags)
    return sentences, labels

train_texts, train_tags = load_and_extract("aae_train.json")
test_essays_texts, test_essays_tags = load_and_extract("aae_test.json")
test_med_texts, test_med_tags = load_and_extract("abstrct_neoplasm_test.json")

# Création d'un dictionnaire qui compte la fréquence des labels pour chaque mot
word_to_tag = {}
for words, tags in zip(train_texts, train_tags):
    for w, t in zip(words, tags):
        w = w.lower()
        if w not in word_to_tag:
            word_to_tag[w] = Counter()
        word_to_tag[w][t] += 1

naive_model = {w: c.most_common(1)[0][0] for w, c in word_to_tag.items()}


___
##**Prédictions**
Dans cette cellule on va transformer les séquence de mot en prédiction d'arguments. Sa particularité est d'intégrer une règle de lissage structurel pour garantir que les résultats respectent les contraintes du format BIO.

Le modèle parcourt la phrase et attribue à chaque mot le label le plus fréquent trouvé lors de l'entraînement. Pour les mots absents du dictionnaire, l'étiquette par défaut est `O`.

Pour éviter les prédictions invalides (comme un label de continuation `I-` qui apparaîtrait seul), la fonction vérifie chaque transition. S'il y a une incohérence, elle force le passage à un label de début `B-`. Cette étape de "nettoyage" permet de reconstruire des segments complets et cohérents, rendant la baseline compatible avec une évaluation stricte.

In [22]:
def predict(texts):
    """
    Réalise la prédiction des étiquettes BIO sur un corpus de textes.

    Args:
        texts (list): Liste de listes de tokens (phrases).

    Returns:
        list: Liste de listes d'étiquettes prédites.
    """
    all_preds = []
    for sentence in texts:
        preds = []
        prev_tag = "O"
        for word in sentence:
            tag = naive_model.get(word.lower(), "O")

            # Un label "I-" ne peut pas suivre un label "O" ou un type différent
            if tag.startswith("I-"):
                tag_type = tag.split("-")[1]
                if prev_tag == "O" or tag_type not in prev_tag:
                    tag = "B-" + tag_type

            preds.append(tag)
            prev_tag = tag
        all_preds.append(preds)
    return all_preds

___
##**Evaluation des Performances**

Cette étape permet de mesurer l'efficacité de la baseline sur les deux domaines de test via le script d'évaluation fournis.

En complément, nous utilisons la bibliothèque `seqeval` pour obtenir une évaluation détaillée, par label, et affiner notre analyse des performances.

In [23]:
def export_for_script(original_file, preds, output_name):
    """
    Formate et exporte les prédictions au format JSON compatible avec le script d'évaluation (evaluate.py).

    Args:
        original_file (str): Nom du fichier source dans le répertoire data.
        preds (list): Liste des séquences d'étiquettes prédites.
        output_name (str): Nom du fichier JSON de sortie à générer.
    """
    path = data_dir / original_file
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    idx = 0
    for doc in data:
        if 'spans' in doc: del doc['spans']
        for paragraph in doc["tokens"]:
            if not paragraph: continue
            current_sent_preds = preds[idx]
            for i, token in enumerate(paragraph):
                if i < len(current_sent_preds):
                    token["arg"] = current_sent_preds[i]
            idx += 1

    with open(output_name, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"\nFichier '{output_name}' généré.")

export_for_script("aae_test.json", preds_essays, "naive_essays_final.json")
export_for_script("abstrct_neoplasm_test.json", preds_med, "naive_med_final.json")

print("--- EVALUATION SCRIPT : ESSAIS ---")
!python evaluate.py data/aae_test.json naive_essays_final.json

print("\n--- EVALUATION SCRIPT : MÉDICAL ---")
!python evaluate.py data/abstrct_neoplasm_test.json naive_med_final.json


Fichier 'naive_essays_final.json' généré.

Fichier 'naive_med_final.json' généré.
--- EVALUATION SCRIPT : ESSAIS ---


********************** SPANS *************************** 
   STRICT EVALUATION
    > Argument mining spans (unlabeled)
      Precision : 0.006026514373400764 
      Recall    : 0.0015998625870651054
      F-score   : 0.002487296305431143
    > Argument mining spans (labeled)
      Precision : 0.00221530634588596 
      Recall    : 0.0006877923492302948 
      F-score   : 0.0010359709458027475

    RELAXED EVALUATION (α = 0.5)
    > Argument mining spans (unlabeled)
      Precision : 0.7378207072066149 
      Recall    : 0.6923472091381503
      F-score   : 0.7104404133787858
    > Argument mining spans (labeled)
      Precision : 0.6443957102422502 
      Recall    : 0.45312841044118274 
      F-score   : 0.5277766738653777



******************* RELATIONS *************************** 
   STRICT EVALUATION
    > Argument mining spans (unlabeled)
      Precision : 1.0 


In [24]:
# Evaluation
print("\n--- ESSAIS ---")
preds_essays = predict(test_essays_texts)
print(classification_report(test_essays_tags, preds_essays, zero_division=0))

print("\n--- MÉDICAL ---")
preds_med = predict(test_med_texts)
print(classification_report(test_med_tags, preds_med, zero_division=0))


--- ESSAIS ---
              precision    recall  f1-score   support

       Claim       0.00      0.00      0.00       301
  MajorClaim       0.00      0.00      0.00       151
     Premise       0.01      0.09      0.02       799

   micro avg       0.01      0.06      0.02      1251
   macro avg       0.00      0.03      0.01      1251
weighted avg       0.01      0.06      0.01      1251


--- MÉDICAL ---
              precision    recall  f1-score   support

       Claim       0.00      0.00      0.00        50
  MajorClaim       0.00      0.00      0.00         5
     Premise       0.00      0.00      0.00        99

   micro avg       0.00      0.00      0.00       154
   macro avg       0.00      0.00      0.00       154
weighted avg       0.00      0.00      0.00       154

